In [1]:
import AdventOfCode

In [2]:
import qualified Data.Vector as V

input <- V.fromList . map V.fromList <$> dayLinesString 18

In [3]:
getPos grid (x,y) = (grid V.! y) V.! x
neighbours grid (x,y) = let
    potential = [(x-1, y-1), (x, y-1), (x+1, y-1),
                 (x-1, y),             (x+1, y),
                 (x-1, y+1), (x, y+1), (x+1, y+1)]
    filtered = filter (\(x,y) -> x >= 0 && x < length (V.head grid) && y >= 0 && y < length grid) potential
    in map (getPos grid) filtered

In [4]:
transition grid (x,y) = let
    current = getPos grid (x,y)
    surround = neighbours grid (x,y)
    in case current of
        '.' -> if count '|' surround >= 3 then '|' else '.'
        '|' -> if count '#' surround >= 3 then '#' else '|'
        '#' -> if count '#' surround > 0 && count '|' surround > 0 then '#' else '.'
    where count c s = length $ filter (==c) s

In [5]:
step :: V.Vector (V.Vector Char) -> V.Vector (V.Vector Char)
step grid = let
    xs = [0..(length (V.head grid))-1]
    ys = [0..(length grid)-1]
    in V.fromList $ map (V.fromList . (\y -> map (\x -> transition grid (x,y)) xs)) ys

In [6]:
result = iterate step input !! 10

In [7]:
wooded = sum . V.map (length . V.filter (=='|'))
yards = sum . V.map (length . V.filter (=='#'))

yards result * wooded result

531417

In [8]:
import qualified Data.Set as Set

findDuplicate :: Ord a => [a] -> Set.Set a -> Either (a, Set.Set a) (Set.Set a)
findDuplicate []     set = Right set
findDuplicate (a:as) set = if Set.member a set
    then Left (a, set)
    else findDuplicate as (Set.insert a set)

In [9]:
Left dup = findDuplicate (iterate step input) Set.empty

In [10]:
firstRepeat = Set.size $ snd dup

In [11]:
Left dup' = findDuplicate (iterate step (fst dup)) Set.empty
cycleLength = Set.size $ snd dup'

In [12]:
result' = iterate step (fst dup) !! ((1000000000-firstRepeat) `mod` cycleLength)

yards result' * wooded result'

205296